# Decision Tree From Scratch - ID3 (Iterative Dichotomiser 3)

### Decision Tree
- Supervised Learning (Classification).
- Non-linear Classifier.

### For more info please check:
- <a href="https://en.wikipedia.org/wiki/Decision_tree_learning">Decision tree learning</a>
- <a href="https://en.wikipedia.org/wiki/ID3_algorithm">ID3 algorithm</a>

In [1]:
import numpy as np
from scipy import stats

In [2]:
class decision_tree_id3:
    def predict(self, X):
        if (hasattr(self, 'tree')):
            t_predicted = self.__predict(X)
            return t_predicted
        else:
            print('Please run fit in order to be able to use predict')
            
    def __predict(self, X):
        y = np.zeros((len(X)))
        for i,x in enumerate(X):
            y[i] = self.__predict_one(x, self.tree)
        return y
    
    def __predict_one(self, x, tree):
        if (tree['label'] is None):
            if (x[tree['col_ind']] >= tree['val']):
                return self.__predict_one(x, tree['right'])
            else:
                return self.__predict_one(x, tree['left'])
        else:
            return tree['label']
    
    def accuracy(self, y_actual, y_predicted):
        return np.mean(y_actual == y_predicted) * 100
    
    def __entropy(self, y):
        distinct_vals = np.unique(y)
        entropy = 0.0
        for distinct_val in distinct_vals:
            p = len(y[y == distinct_val]) / len(y)
            entropy -= p * np.log2(p)
        return entropy
    
    def __att_selection(self, X, y, used_attributes):
        parent_entropy = self.__entropy(y)
        max_gain = 0.0
        split_col_ind = None
        split_val = None
        
        for col_ind, col_vec in enumerate(X.T):
            if (bool(np.isin(col_ind, used_attributes))):
                continue
            distinct_col_values = np.unique(col_vec)
            for val in distinct_col_values:
                y_right = y[col_vec >= val]
                y_left = y[col_vec < val]
                
                split_weighted_entropy = (len(y_right)/len(y)) * self.__entropy(y_right)
                split_weighted_entropy += (len(y_left)/len(y)) * self.__entropy(y_left)
                info_gain = parent_entropy - split_weighted_entropy
                
                if (info_gain > max_gain):
                    max_gain = info_gain
                    split_col_ind = col_ind
                    split_val = val
                    
        return split_col_ind, split_val
    
    def __fit_tree(self, X, y, used_attributes):
        if (len(np.unique(y)) == 1):
            return {'col_ind': None,
                    'attribute': None,
                    'val': None,
                    'right': None,
                    'left': None,
                    'label': y[0]}
        
        split_col_ind, split_val = self.__att_selection(X, y, used_attributes)
        split_attribute = self.attributes[split_col_ind]
        
        used_attributes = np.append(used_attributes, split_col_ind)
                
        if (split_col_ind is None or split_val is None):
            return {'col_ind': None,
                    'attribute': None,
                    'val': None,
                    'right': None,
                    'left': None,
                    'label': stats.mode(y)[0][0]}
        
        right_X = X[X[:, split_col_ind] >= split_val]
        right_y = y[X[:, split_col_ind] >= split_val]
        
        left_X = X[X[:, split_col_ind] < split_val]
        left_y = y[X[:, split_col_ind] < split_val]
        
        right_tree = self.__fit_tree(right_X, right_y, used_attributes)
        left_tree = self.__fit_tree(left_X, left_y, used_attributes)
        
        return {'col_ind': split_col_ind,
                'attribute': split_attribute,
                'val': split_val,
                'right': right_tree,
                'left': left_tree,
                'label': None}        
        
    def fit(self, X, t, attributes = []):
        self.X = X
        self.t = t
        self.attributes = attributes
        self.tree = {}
        used_attributes = []
        
        if (len(self.attributes) == 0 and len(X[0]) != 0):
            self.attributes = np.array(['X{}'.format(i+1) for i in range(len(X[0]))])
            
        elif (len(self.attributes) != len(X[0])):
            print('Please make sure that the attrebutes length equals to number of columns of data')
            return
        
        self.tree = self.__fit_tree(self.X, self.t, used_attributes)
        

# Dataset 1

### Training

In [3]:
Data = np.genfromtxt('synth.tr.csv', delimiter=',', skip_header=True)
X = Data[:, 1:3]
t = Data[:, 3]

dt = decision_tree_id3()
dt.fit(X, t)

### Testing

In [4]:
Data_test = np.genfromtxt('synth.te.csv', delimiter=',', skip_header=True)
X_test = Data_test[:, 1:3]
y_actual = Data_test[:, 3]
y_predicted = dt.predict(X_test)
acc = dt.accuracy(y_actual, y_predicted)
print('Accuracy', acc, '%')

Accuracy 90.0 %


In [5]:
dt.tree

{'col_ind': 1,
 'attribute': 'X2',
 'val': 0.49785904,
 'right': {'col_ind': 0,
  'attribute': 'X1',
  'val': -0.59823393,
  'right': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 1.0},
  'left': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0.0},
  'label': None},
 'left': {'col_ind': 0,
  'attribute': 'X1',
  'val': -0.36528923,
  'right': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0.0},
  'left': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0.0},
  'label': None},
 'label': None}

# Dataset 2

### Training

In [6]:
Data2 = np.genfromtxt('Data1.txt')
# Data2 = Data2[Data2[:,0] < 40] # Without outliers
X2 = Data2[:, 0:2]
t2 = np.array([1 if i >= 0 else 0 for i in Data2[:, 2]])

dt2 = decision_tree_id3()
dt2.fit(X2, t2)

### Testing

In [7]:
Data2_test = np.genfromtxt('Test1.txt')
X2_test = Data2_test[:, 0:2]
y2_actual = np.array([1 if i >= 0 else 0 for i in Data2_test[:, 2]])
y2_predicted = dt2.predict(X2_test)
acc2 = dt2.accuracy(y2_actual, y2_predicted)
print('Accuracy', acc2, '%')

Accuracy 97.5 %


In [8]:
dt2.tree

{'col_ind': 1,
 'attribute': 'X2',
 'val': 3.6454,
 'right': {'col_ind': 0,
  'attribute': 'X1',
  'val': 0.07623,
  'right': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 1},
  'left': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0},
  'label': None},
 'left': {'col_ind': 0,
  'attribute': 'X1',
  'val': 7.0287,
  'right': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 1},
  'left': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0},
  'label': None},
 'label': None}

# Dataset 3

### Training

In [9]:
from sklearn.datasets import make_classification


In [10]:
X3, t3 = make_classification(n_samples=1000, n_features=2, n_redundant=0, n_informative=1,
                             n_clusters_per_class=1, random_state=14, class_sep=1, n_classes=2)

dt3 = decision_tree_id3()
dt3.fit(X3, t3)

### Testing

In [11]:
X3_test, y3_actual = make_classification(n_samples=300, n_features=2, n_redundant=0, n_informative=1,
                             n_clusters_per_class=1, random_state=14, class_sep=1, n_classes=2)

y3_predicted = dt3.predict(X3_test)
acc3 = dt3.accuracy(y3_predicted, y3_actual)
print('Accuracy', acc3, '%')

Accuracy 98.66666666666667 %


In [12]:
dt3.tree

{'col_ind': 0,
 'attribute': 'X1',
 'val': -0.07071678678023519,
 'right': {'col_ind': 1,
  'attribute': 'X2',
  'val': 1.5037879976276987,
  'right': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 1},
  'left': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 1},
  'label': None},
 'left': {'col_ind': 1,
  'attribute': 'X2',
  'val': 1.0864076374038787,
  'right': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0},
  'left': {'col_ind': None,
   'attribute': None,
   'val': None,
   'right': None,
   'left': None,
   'label': 0},
  'label': None},
 'label': None}